In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from QBUS2820 import rmse_jack, r2_jack 
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from ExtraCode import getResultTable
from sklearn.model_selection import cross_val_score
from QBUS2820 import forward
from sklearn.linear_model import ElasticNet

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module wil

### It's ok...

In [80]:
pred = []
method = []

data = pd.read_csv('Train6_2.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')
y_train = np.log(y_train)


#Forward selection
fwd = forward()
fwd.fit(final_train, y_train)
predforward = fwd.predict(final_test)
method.append('Forward')
pred.append(np.exp(predforward))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost')
pred.append(np.exp(predFinalBoost))

regr = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)
regr = regr.fit(final_train,y_train)
adapred = regr.predict(final_test)
method.append('AdaBoost')
pred.append(np.exp(adapred))

In [81]:
xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
testFinaldmat = xgb.DMatrix(final_test)
xgpred = final_gb.predict(testFinaldmat)
method.append('XGBoost')
pred.append(np.exp(xgpred))

In [82]:
pred2 = (np.exp(xgpred)+np.exp(adapred)+np.exp(predFinalBoost))/3
pred.append(pred2)
method.append('XGBoost, ada, GB')

In [83]:
pred3 = (np.exp(xgpred)+np.exp(predFinalBoost))/2

pred5 = (np.exp(xgpred)+np.exp(adapred))/2
pred.append(pred3)
pred.append(pred5)
method.append('xg GB')
method.append('xg adaboost')

In [84]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(final_train,y_train)

In [85]:

predFinalExtRandomForestlad = regr.predict(final_test)

pred5 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad)+np.exp(predFinalBoost))/3
pred.append(pred5)
method.append('XGBoost, random tree, GB')

In [86]:
pred6 = (np.exp(xgpred)+np.exp(predFinalExtRandomForestlad))/2
pred.append(pred6)
method.append('xg Random tree')

In [87]:
pred10 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad))/3
pred.append(pred10)
method.append('XGBoost, GBoost Random Forest')

In [88]:
#Validation function
n_folds = 10
def RMSLE(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(final_train.values)
    rmse= np.sqrt(-cross_val_score(model, final_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return rmse

In [101]:
#LASSO
level = [0.1, 1, 5, 10, 20, 50, 75, 100]
for x in level:
    lasso = Lasso(alpha = x)
    lasso.fit(final_train, y_train) 
    pred_L = lasso.predict(final_test)
    method.append('LASSO_Alpha')
    pred.append(np.exp(pred_L))



/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [25]:

alphas = [0.1, 1, 5, 10, 20, 50, 75, 100]
for alpha in alphas:
    rg = Lasso(alpha = alpha)
    rg.fit(final_train, y_train)
    rg_score = rg.score(final_train, y_train)
    rmse = RMSLE(rg).mean()
    print("For alpha = {} the Lasso model accuracy score = {:.2f} and  mean rmse = {:.3f}".format(alpha, rg_score*100, rmse))

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 0.1 the Lasso model accuracy score = 96.55 and  mean rmse = 21741.100


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 1 the Lasso model accuracy score = 96.55 and  mean rmse = 21457.697


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

...

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 75 the Lasso model accuracy score = 95.75 and  mean rmse = 18496.744


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:4

For alpha = 100 the Lasso model accuracy score = 95.58 and  mean rmse = 18508.492


/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [105]:
mu=y_train.mean()
sigma=y_train.std() 
standardPrice=(y_train-mu)/sigma

#LASSO
lasso = Lasso(alpha = 1)
lasso.fit(final_train, np.ravel(standardPrice)) 
pred_L = lasso.predict(final_test)
predFinalLasso = (pred_L*sigma) + mu
method.append('LASSO Standard')
pred.append(predFinalLasso)

In [11]:
#RIDGE
#alphas = np.exp(np.linspace(-10,20,500)) 
#ridge = RidgeCV(alphas=alphas, cv=10)
#ridge.fit(final_train, y_train)
#ridge = Ridge(alpha=ridge.alpha_)
#ridge.fit(final_train, y_train)
#pred_R = ridge.predict(final_test)
#method.append('Ridge')
#pred.append(pred_R)

alphas = [0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 75, 100]
for alpha in alphas:
    rg = Ridge(alpha = alpha)
    rg.fit(final_train, y_train)
    rg_score = rg.score(final_train, y_train)
    rmse = RMSLE(rg).mean()
    print("For alpha = {} the Ridge model accuracy score = {:.2f} and  mean rmse = {:.3f}".format(alpha, rg_score*100, rmse))

For alpha = 0.001 the Ridge model accuracy score = 96.55 and  mean rmse = 21717.670
For alpha = 0.01 the Ridge model accuracy score = 96.55 and  mean rmse = 21574.424
For alpha = 0.1 the Ridge model accuracy score = 96.53 and  mean rmse = 20767.142
For alpha = 1 the Ridge model accuracy score = 96.31 and  mean rmse = 19317.776
For alpha = 5 the Ridge model accuracy score = 95.78 and  mean rmse = 18998.876
For alpha = 10 the Ridge model accuracy score = 95.40 and  mean rmse = 19176.510
For alpha = 20 the Ridge model accuracy score = 94.92 and  mean rmse = 19516.232
For alpha = 50 the Ridge model accuracy score = 94.14 and  mean rmse = 20174.935
For alpha = 75 the Ridge model accuracy score = 93.74 and  mean rmse = 20549.666
For alpha = 100 the Ridge model accuracy score = 93.44 and  mean rmse = 20853.185


In [115]:
alpha = 20
rg = Ridge(alpha = alpha)
rg.fit(final_train, y_train)
y_pred_rg = (rg.predict(final_test))
y_pred_rg

pred.append(np.exp(y_pred_rg))
method.append('Ridge alpha 20')

In [107]:
#RIDGE standard 1
alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=10)
ridge.fit(final_train, np.ravel(standardPrice))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(standardPrice))
pred_R = ridge.predict(final_test)
predFinalRidge = (pred_R*sigma) + mu
method.append('Ridge Standard1')
pred.append(predFinalRidge)

In [ ]:
#Ridge standard 2
alphas = np.exp(np.linspace(-10,20,500))

ridge = RidgeCV(alphas=alphas, cv=10)
ridge.fit(final_train, np.ravel(standardPrice))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(standardPrice))
pred_R = ridge.predict(final_test)
predFinalRidge = (pred_R*sigma) + mu
method.append('Ridge Standard1')
pred.append(predFinalRidge)

In [91]:
pred11 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4
pred.append(pred11)
method.append('XGBoost, GBoost Random Forest Ridge')

In [92]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred_L))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest LASSO')

In [93]:
from sklearn.linear_model import ElasticNet

regr = ElasticNet(random_state=0, alpha = 1)
    
regr.fit(final_train, y_train)    
pred13 = regr.predict(final_test)
pred.append(np.exp(pred13))
method.append('ENET')

In [94]:
pred12 = (xgpred+predFinalBoost+predFinalExtRandomForestlad+pred13)/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest ENET')

In [95]:
pred13 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg))/3
pred.append(pred13)
method.append('XGBoost, GBoost Ridge')

In [96]:
pred14 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward))/4
pred.append(pred14)
method.append('XGBoost, GBoost Ridge Forward')

In [97]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(predFinalExtRandomForestlad))/5
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward Tree')

In [ ]:
pred16 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward)+ np.exp(predFinalExtRandomForestlad))/5
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward Lasso')

In [116]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
Forward,23834.417,4933.468,0.884,0.047,14042.658,0.884
GBoost,20918.845,3335.212,0.911,0.019,13764.739,0.911
AdaBoost,26396.260,3445.265,0.858,0.023,18521.883,0.858
XGBoost,20930.701,3433.914,0.910,0.020,13937.653,0.910
"XGBoost, ada, GB",20990.748,3686.173,0.910,0.022,13907.523,0.910
xg GB,20256.938,3491.853,0.916,0.020,13369.483,0.916
xg adaboost,22014.324,3690.781,0.901,0.023,14815.195,0.901
"XGBoost, random tree, GB",21014.092,3760.257,0.910,0.022,13862.684,0.910
xg Random tree,21990.427,3808.881,0.901,0.023,14686.106,0.901
"XGBoost, GBoost Random Forest",21014.092,3760.257,0.910,0.022,13862.684,0.910


In [32]:

def getResultTable(rows, predictions):
    columns=['Test RMSE', 'SE', 'Jack R2', 'SE', 'MAE', 'R-square']
    results=pd.DataFrame(0.0, columns=columns, index=rows)

    for row,pred in zip(range(0,len(rows)),predictions):
        results.iloc[row,0], results.iloc[row,1] = rmse_jack(y_test, pred)
        results.iloc[row,2], results.iloc[row,3] = (r2_jack(y_test, pred))
        results.iloc[row,4] = mean_absolute_error(y_test, pred)
        results.iloc[row,5] = r2_score(y_test,pred)
    return results.round(3)

# Kaggle Predictions

In [117]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [118]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [119]:
#Extremely Random forest
regr = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr = regr.fit(data,y_price)
predFinalExtRandomForestlad = regr.predict(kaggle)

In [120]:

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [121]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [122]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(predFinalExtRandomForestlad)+np.exp(y_pred_rg))/4

In [123]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day6_4.csv", index=False)

### 

## Does worse

In [21]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [22]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [23]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [24]:
alpha = 1
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [25]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(y_pred_rg))/3

In [26]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day7_1.csv", index=False)

In [59]:
data = pd.read_csv('Train6_2.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_2.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [60]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [61]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [62]:
alpha = 1
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [56]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(y_pred_rg))/3

In [57]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day7_2.csv", index=False)

In [66]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [67]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [68]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [69]:
alpha = 1
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [73]:
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)

In [74]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(y_pred_rg)+np.exp(predforward))/4

In [75]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day7_3.csv", index=False)

In [124]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [125]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [126]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [127]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [128]:
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)

In [129]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day7_4.csv", index=False)

In [135]:
data = pd.read_csv('Train6_1.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_1.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [136]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
prediction = final_gb.predict(testFinaldmat)

In [137]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [143]:
alpha = 5
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [140]:
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)

In [144]:
finalprediction = (np.exp(predFinalBoost)+np.exp(prediction)+np.exp(y_pred_rg)+np.exp(predforward))/4

In [145]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction5 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction5.to_csv("Day7_5.csv", index=False)